In [ ]:
cd /p/home/jusers/todorov3/jusuf/bmp_code

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
from datetime import datetime
import pingouin as pg
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pingouin import ttest

import pingouin as pg
from joblib import Parallel, delayed
from itertools import product
import warnings

from bmp_config import path_data, envcode2env
from bmp_behav_proc import *
load = 1
if load:
    fnf_fhl = pjoin(path_data,'dfcs_fixhistlen.pkl')
    print(fnf_fhl)
    print( str(datetime.fromtimestamp(os.stat(fnf_fhl).st_mtime)))
    dfcs_fixhistlen = pd.read_pickle(fnf_fhl )
else:
    fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
    print(fnf)
    print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
    df_all_multi_tsz = pd.read_pickle(fnf)
    df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                            ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
    df,dfall,ES_thr,envv,pert = addBehavCols2(df);
    dfc = df.copy()
    dfcs,dfcs_fixhistlen,dfcs_fixhistlen_untrunc,histlens  = addWindowStatCols(dfc, ES_thr, 
                                        varn0s = ['error','error_pscadj','error_pscadj_abs'])
    dfcs_fixhistlen, me_pct_excl = truncLargeStats(dfcs_fixhistlen_untrunc, histlens, 5.)
import pingouin as pg
dfcs_fixhistlen['environment'] = dfcs_fixhistlen['environment'].astype(int)
df_ = dfcs_fixhistlen[['environment','subject','trials','error_pscadj_abs_Tan29']]
assert not df_.duplicated(['subject','trials']).any()
all_suffixes = 'mav,std,invstd,mavsq,mav_d_std,mav_d_var,Tan,invmavsq,invmav,std_d_mav,invTan'.split(',')

dfcs_fixhistlen0 = dfcs_fixhistlen.copy()
print(len(dfcs_fixhistlen))
c = ~  np.isinf(dfcs_fixhistlen['err_sens']) 
dfcs_fixhistlen = dfcs_fixhistlen.loc[c]
print(len(dfcs_fixhistlen))
c2 = dfcs_fixhistlen['err_sens'] > -41
dfcs_fixhistlen = dfcs_fixhistlen.loc[c2]
print(len(dfcs_fixhistlen))

# make sure length n  is inside
varn0 = 'error_pscadj'
n = 3
for suffix in all_suffixes:
    s = f'{varn0}_{suffix}{n}'
    if s not in dfcs_fixhistlen.columns:
        print(s, 'not in dfcs_fixhistlen.columns')
    assert s in dfcs_fixhistlen.columns

print('all_suffixes = ', all_suffixes )

## Plot of distributions (Fig S6)

## signed error

In [ ]:
#pooled
%matplotlib inline
from figure.plots import plot_ES_vs_Tan
plot_ES_vs_Tan(dfcs_fixhistlen, subjects_set='all', coln='error_pscadj_Tan20')
plt.show()

In [ ]:
# single
%matplotlib inline
from figure.plots import plot_ES_vs_Tan

coln2xlim = { 'error_pscadj_Tan20':  
              {'stable': (-0.2, 10.5), 'random': (-0.05, 1)}} 
plot_ES_vs_Tan(dfcs_fixhistlen, subjects_set='one', hist_bins = (20,20), 
               coln='error_pscadj_Tan20', coln2xlim=coln2xlim,
               legend_loc=  (0.95, 0.65))
plt.show()

## Absolut error

In [ ]:
#pooled
%matplotlib inline
from figure.plots import plot_ES_vs_Tan
plot_ES_vs_Tan(dfcs_fixhistlen, subjects_set='all', coln='error_pscadj_abs_Tan20')
plt.show()

In [ ]:
# single
%matplotlib inline
from figure.plots import plot_ES_vs_Tan

coln2xlim = { 'error_pscadj_abs_Tan20':  
              {'stable': (-0.2, 10.5), 'random': (0.0, 5)}} 
plot_ES_vs_Tan(dfcs_fixhistlen, subjects_set='one', hist_bins = (20,20), 
               coln='error_pscadj_abs_Tan20', coln2xlim=coln2xlim,
               legend_loc=  (0.97, 0.88))
plt.show()